In [3]:
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
import scipy.stats as stats
import keras
import bottleneck as bn
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


**To reproduce the statistics presented in the result section, please make sure to import all the dataset and weight files in your working directory and define all the functions below.**

In [11]:
random_seed=0
np.random.seed(random_seed)
#random.seed(random_seed)
tf.random.set_seed(random_seed)
sns.set(rc={'figure.figsize':(10,8)})

In [12]:
"""
input:  
mask: known mask (numpy matrix)
values: known entries (numpy matrix)
shape_mat: shape of the returned matrix (tuple)
output: 
mat: matrix with known values at the known indices and zero elsewhere (numpy matrix)
"""
def mask_to_mat(mask,values,shape_mat):
    mat=np.zeros_like(shape_mat)
    mat[mask]=np.squeeze(np.array(values),axis=0)
    return mat

In [13]:

'''
input:
mat: matrix to binarise
output:
bin_mat:binarised matrix 
'''
def binarise_mat(mat):
    bin_mat=np.zeros(mat.shape)
    bin_mat[mat>=4]=1
    bin_mat[mat<4]=0
    return bin_mat

In [14]:


class VAE(tf.keras.Model):
    def __init__(self,latent_dim,n,random_seed,beta,drop_out_rate,soft_max,correction):
        super(VAE,self).__init__()
        #Initialising the hyperparameters
        self.latent_dim=latent_dim
        self.random_seed=random_seed
        self.n=n
        self.beta=beta
        self.drop_out_rate=drop_out_rate
        self.soft_max=soft_max
        #Initialising dense layers
        self.inf_dense_600=tf.keras.layers.Dense(600,activation=tf.nn.relu,kernel_initializer='random_normal')#kernel_initializer=initializer)
        self.inf_dense_latent=tf.keras.layers.Dense(2*latent_dim,activation=None,kernel_initializer='random_normal')#kernel_initializer=initializer)
        self.gen_dense_600=tf.keras.layers.Dense(600,activation=tf.nn.relu,kernel_initializer='random_normal')#kernel_initializer=initializer)
        self.gen_dense_n=tf.keras.layers.Dense(self.n,activation=None,kernel_initializer='random_normal')#kernel_initializer=initializer)
        #Initialising dropout layers
        self.drop_out=tf.keras.layers.Dropout(rate=self.drop_out_rate,seed=self.random_seed)

        self.correction=correction
        self.correct_coeff=None
        self.miu=None
        self.logvar=None
        self.z_u=None
        self.prob=None
        self.logits=None
        self.neg_ll=None
        self.KL=None
        self.neg_ll_to_include=None
        
    def inference_net(self,input_val):
        _,n=input_val.shape
    
        #dense inference layer [n → 600]
        h=self.inf_dense_600(input_val)
        if eval==False:
            h=self.drop_out(h)
        #dense inference layer [600 → 2*latent_dims]
        h=self.inf_dense_latent(h)
        #selecting the first half of the entries to be the mean of variational distribution
        self.miu=h[:,:self.latent_dim]
        #selecting the remaining entries to be the log variance of variational distribution
        self.logvar=h[:,self.latent_dim:]  
        
    def reparametrisation_trick(self):
        """
         input:
            self.miu:mean of the variational distribution || batch size x latent dimension
        output:
            self.z_u:sampled latent variable || batch size x latent dimension 
         """
        epis=tf.random.normal(tf.shape(self.miu),mean=0,stddev=1,seed=self.random_seed)
        self.z_u=epis*tf.exp(0.5*self.logvar)+self.miu
    
    
    def generation_net(self):
        #dense generation layer [latent_dims → 600]
        h=self.gen_dense_600(self.z_u)
        #dense generation layer [600 → n] 
        if eval==False:
            h=self.drop_out(h)
        h=self.gen_dense_n(h)
        #Saving the unormalised multinomial probability
        self.logits=h
        if self.soft_max==True:
            self.prob=tf.nn.softmax(h)
        else:
          #applying sigmoid activation to generate probability of an item being one 
          self.prob=tf.nn.sigmoid(h)
        
    
    def forward_pass(self,input_val):
        self.inference_net(input_val)
        self.reparametrisation_trick()
        self.generation_net()
        return self.prob

    def prediction(self,input_val):
        self.inference_net(input_val)
        self.z_u=self.miu
        self.generation_net()
        return self.prob

    def call(self,input_val):
        self.forward_pass(input_val)
        return self.prob
    
    
    def ELBO(self,input_val,known_mask_batch):
        """ input:
                miu:mean of the variational distribution || batch size x latent dimension
                logvar:log of the variance of the variational distribution || batch size x latent dimension
                log_p: log multinomial probability distribution over items || batch size x n
                input_val:input data || batch size x n 
            output:
                ELBO: scalar
        """ 
        batch_size=self.prob.shape[0]
        if self.correction==True:
            self.correct_coeff=train_in[known_mask].sum()/known_mask.sum()
            self.neg_ll=-((1-self.correct_coeff)*input_val*tf.math.log(1e-7+self.prob)+self.correct_coeff*(1-input_val)*tf.math.log(1e-7+1-self.prob))
        else:
            self.neg_ll=-(input_val*tf.math.log(1e-7+self.prob)+(1-input_val)*tf.math.log(1e-7+1-self.prob))

        self.neg_ll_to_include=tf.reduce_sum(self.neg_ll[known_mask_batch])
      
        self.neg_11_to_include=(1/batch_size)*self.neg_ll_to_include
        
        self.KL=tf.reduce_sum(-0.5*(2*self.logvar-tf.exp(self.logvar)-tf.square(self.miu)+1),axis=1)

        ELBO=self.neg_ll_to_include+tf.reduce_mean(self.beta*self.KL)
        return ELBO
    
        
        
        

In [29]:
#[1]
def recallatN(N,mat_to_eval,predicted,user_level=False): 
    """
  input:
    N：length of the recommendation list
    mat_to_eval:ground truth matrix 
    predicted:output of the generation network (probability over items)
    user_level:if true output the metric score for each user
    
  output:
    self.z_u:sampled latent variable || batch size x latent dimension 
    """
    val_size=predicted.shape[0]
  #sorting the unormalised multinomial probabilities over items
  #such that the first K elements correspond to the largest K elements 
    sorted_idx=bn.argpartition(-predicted,kth=N)
  #selecting the index of the top k elements
    top_N_idx=sorted_idx[:,:N]
  #creating a boolean matrix of the same shape as the held-out batch
    x_pred=np.zeros_like(mat_to_eval,dtype=bool)
  #assigning true values for the predicted top k elements 
    x_pred[np.arange(val_size)[:,np.newaxis],top_N_idx]=True
  #creating a bolean array that selects the positive entries in the held-out set
    val_positive_bool=mat_to_eval>0
  #counting the number of true positive items in the top-k list
    correct_positive=np.logical_and(x_pred,val_positive_bool).sum(axis=1).astype(np.float32)
  #mean recall for the users in held-out batch
    if user_level==True:
        to_return=remove_nan(correct_positive/np.minimum(N,mat_to_eval.sum(axis=1)))
    else:
        to_return=np.nanmean(correct_positive/np.minimum(N,mat_to_eval.sum(axis=1)))
    return to_return

In [30]:
def binarise(mat):
    '''
    input:
    mat:matrix with probability values to binarise (numpy matrix/array)
    output:
    bin_mat:binarised matrix with same shape as the input matrix (numpy matrix/array)
    '''
    bin_mat=np.zeros_like(mat)
    bin_mat[mat>=0.5]=1
    return bin_mat

In [31]:

def recon_acc(predicted,mat_to_eval,eval_mask):
  '''
  input: 
  predicted:output of the generation network
  mat_to_eval:ground truth matrix
  eval_mask:boolean mask that select entries for evaluation
  output:
  reconstruction accurary score for the entries under evaluation
  '''
  return (binarise(predicted[eval_mask])==mat_to_eval[eval_mask]).sum()/mat_to_eval[eval_mask].shape[1]

In [32]:
  '''
  input:
  predicted:output of the generation network
  eval_entries:ground truth entries
  eval_bool: boolean mask that select entries for evaluation
  output:
  TP:no. of true positives
  TN:no. of true negatives
  FP:no. of false positives
  FN:no. of false negatives
  '''
def compute_metrics(predicted,eval_entries,eval_bool):
    TP=predicted[eval_bool][eval_entries==1].sum()
    TN=(predicted[eval_bool][eval_entries==0]==0).sum()
    FP=(predicted[eval_bool][eval_entries==0]==1).sum()
    FN=(predicted[eval_bool][eval_entries==1]==0).sum()
    return TP,TN,FP,FN

In [34]:
def remove_nan(list):
    '''
    input:
    list:list of values 
    list:list with nan values removed
    '''
    return np.array(list)[~np.isnan(list)]

In [35]:
'''
Function that name each experiment depending on the evaluation mode specified by the boolean parameters
'''
def name_experiment(hard_gen,soft_max,correction,sampling,pretrained=False):
    name=[]
    if hard_gen==True:
        name.append("hard_gen_")
    else:
        name.append("easy_gen_")
    if soft_max==True:
        name.append("soft_max_")
    else:
        name.append("sigmoid_")
    if correction==True:
        name.append("corrected_")
    else:
        name.append("nocorrection_")
    if sampling==True:
        name.append("sampled")
    else:
        name.append("no_sampling")
    if pretrained==True:
        name.append("_pretrained")
    if pretrained==False:
        name=name[0]+name[1]+name[2]+name[3]
    else:
        name=name[0]+name[1]+name[2]+name[3]+name[4]
    print("Evaluation Mode: "+name)  
    return name

In [36]:
'''
Function that load train, test, and validation dataset for easy/hard generation task with/without sampling
'''
def train_loader(hard_gen,sampling):
    if hard_gen==False:
        if sampling==False:
            train_in=sp.load_npz(os.getcwd()+"/tr_implicit_nosampling_easy.npz").todense()
            known_mask=sp.load_npz(os.getcwd()+"/known_mask_nosampling_easy.npz").todense()
        else:
            train_in=sp.load_npz(os.getcwd()+"/tr_implicit_sampled_easy.npz").todense()
            known_mask=sp.load_npz(os.getcwd()+"/known_mask_sampled_easy.npz").todense()
    
    else:
        if sampling==False:
            train_in=sp.load_npz(os.getcwd()+"/tr_implicit_nosampling_hard.npz").todense()
            known_mask=sp.load_npz(os.getcwd()+"/known_mask_nosampling_hard.npz").todense()
        else:
            train_in=sp.load_npz(os.getcwd()+"/tr_implicit_sampled_hard.npz").todense()
            known_mask=sp.load_npz(os.getcwd()+"/known_mask_sampled_hard.npz").todense()
    return train_in,known_mask


def test_val_loader(hard_gen,train_in):
    if hard_gen==True:
        val_fold_in=sp.load_npz(os.getcwd()+"/val_fold_in_implicit.npz").todense()
        val_report=sp.load_npz(os.getcwd()+"/val_report_implicit.npz").todense()
        val_fold_in_bool=sp.load_npz(os.getcwd()+"/val_fold_in_bool.npz").todense()
        val_report_bool=sp.load_npz(os.getcwd()+"/val_report_bool.npz").todense()

        test_fold_in=sp.load_npz(os.getcwd()+"/test_fold_in_implicit.npz").todense()
        test_report=sp.load_npz(os.getcwd()+"/test_report_implicit.npz").todense()
        test_fold_in_bool=sp.load_npz(os.getcwd()+"/test_fold_in_bool.npz").todense()
        test_report_bool=sp.load_npz(os.getcwd()+"/test_report_bool.npz").todense()
        return val_fold_in,val_report,val_fold_in_bool,val_report_bool,test_fold_in,test_report,test_fold_in_bool,test_report_bool

    else:
        val_entries=sp.load_npz(os.getcwd()+"/val_entries_implicit_easy.npz").todense()
        test_entries=sp.load_npz(os.getcwd()+"/test_entries_implicit_easy.npz").todense()
        val_bool=sp.load_npz(os.getcwd()+"/val_bool_easy.npz").todense()
        test_bool=sp.load_npz(os.getcwd()+"/test_bool_easy.npz").todense()
        val_mat=mask_to_mat(val_bool,val_entries,train_in)
        test_mat=mask_to_mat(test_bool,test_entries,train_in)
        return val_entries,test_entries,val_bool,test_bool,val_mat,test_mat
    

In [38]:
def metrics_eval(TP,TN,FN,FP):
    '''
    input:
    TP：no. of true positives
    TN: no. of true negatives
    FN：no. of false negatives
    FP: no. of false positives
    output:
    recall_val:a single recall score based on all users
    neg_recall:a single specificity score based on all users
    hit_val: a single precision score based on all users
    '''
    recall_val=TP/(TP+FN)
    neg_recall_val=TN/(TN+FP)
    hit_val=TP/(TP+FP)
    return recall_val,neg_recall_val,hit_val


In [39]:
'''
Function that returns test recall, specificity and precision for VAE-sigmoid and recall@20,recall@50,recall@100 for VAE-softmax
'''
def load_pred(name,mf=False,hard_gen=False,soft_max=False):

    if mf==False:
        vae=VAE(parameters.get("latent_dim"),train_in.shape[1],random_seed,parameters.get("beta"),parameters.get("drop_out_rate"),soft_max,correction)
        vae.build(train_in.shape)
        vae.load_weights(os.getcwd()+'/diff_init2_'+name+".h5")
        eval=True
        if hard_gen==False:
            predicted=vae.prediction(train_in)
        else:
            predicted=vae.prediction(test_fold_in)
    else:
        predicted=np.array(sp.load_npz(os.getcwd()+"/predicted_mf.npz").todense())

    if soft_max==False:
        if hard_gen==False:
            recall_list,neg_recall_list,hit_list=user_level_metrics(binarise(predicted),np.array(test_mat),np.array(test_bool),np.array(test_entries))
        else:
            recall_list,neg_recall_list,hit_list=user_level_metrics(binarise(predicted),np.array(test_report),np.array(test_report_bool),np.array(test_report[test_report_bool]))
        print("Test recall:{}".format(recall_list.mean()))
        print("Test specificity:{}".format(neg_recall_list.mean()))
        print("Test precision:{}".format(hit_list.mean()))
    else:
        recall_20_list=recallatN(20,test_report,predicted,user_level=True)
        recall_50_list=recallatN(50,test_report,predicted,user_level=True)
        recall_100_list=recallatN(100,test_report,predicted,user_level=True)
        print("recall@20:{}".format(recallatN(20,test_report,predicted)))
        print("recall@50:{}".format(recallatN(50,test_report,predicted)))
        print("recall@100:{}".format(recallatN(100,test_report,predicted)))

    if soft_max==False:
        if mf==False:
            return recall_list,neg_recall_list,hit_list,vae
        else:
            return recall_list,neg_recall_list,hit_list
    else:
        return recall_20_list,recall_50_list,recall_100_list


In [40]:
'''
Function that loads TN,TP,FN,FP given an experiment name for VAE-sigmoid and recall@20 for VAE-softmax
'''
def load_metrics(name,soft_max=False):
    if soft_max==False:
        TN=np.load(os.getcwd()+"/TN_list_"+name+".npy")
        TP=np.load(os.getcwd()+"/TP_list_"+name+".npy")
        FN=np.load(os.getcwd()+"/FN_list_"+name+".npy")
        FP=np.load(os.getcwd()+"/FP_list_"+name+".npy")
        return TN,TP,FN,FP
    else:
        recallat20=np.load(os.getcwd()+"/recallat20"+name+".npy")
        return recallat20
  

In [43]:
def user_level_metrics(predicted,mat,eval_bool,eval_entries):
    '''
    input:
    predicted:output of the generation network
    mat:ground truth matrix
    eval_bool:boolean mask that return the values for evaluation
    eval_entries: ground truth values 
    output:
    list of per user recall value with nan values removed
    list of per user specificity value with nan values removed
    list of per user precision value with nan values removed
    '''
    recall_list=[]
    hit_list=[]
    neg_recall_list=[]

    for i in range(predicted.shape[0]):
        recall_list.append(predicted[i][mat[i]==1].sum()/(mat[i]==1).sum())
        #recon_acc_list.append((predicted[i][eval_bool[i]]==mat[i][eval_bool[i]]).sum()/mat[i][eval_bool[i]].shape[0])
        neg_recall_list.append((((predicted[i][eval_bool[i]])[mat[i][eval_bool[i]]==0])==0).sum()/(mat[i][eval_bool[i]]==0).sum())
        hit_list.append(np.logical_and(predicted[i][eval_bool[i]]==1,mat[i][eval_bool[i]]==1,mat[i][eval_bool[i]]==1).sum()/((predicted[i][eval_bool[i]])==1).sum())
    return remove_nan(recall_list),remove_nan(neg_recall_list),remove_nan(hit_list)

In [45]:
'''
Function that performs paired sample t-test for user-level recall,specificity for VAE-sigmoid, and user-level recall@20,50,100 for VAE-softmax
independent t-test is performed for precision 
'''
def comparison(list11,list12,list21,list22,list31,list32,soft_max=False):
    if soft_max==False:
        print("recall: {}".format(stats.ttest_rel(list11,list12)))
        print("specificity: {}".format(stats.ttest_rel(list21,list22)))
        print("precision: {}".format(stats.ttest_ind(list31,list32)))

    else:
        print("recall@20: {}".format(stats.ttest_rel(list11,list12)))
        print("recall@50: {}".format(stats.ttest_rel(list21,list22)))
        print("recall@100: {}".format(stats.ttest_rel(list31,list32)))




In [46]:
'''
Function that names and load the train,validation and test set for a given evaluation mode
'''
def name_load(hard_gen,soft_max,sampling,correction,pretrained=False):
    name=name_experiment(hard_gen,soft_max,correction,sampling,pretrained)
    train_in,known_mask=train_loader(hard_gen,sampling)
    if hard_gen==False:
        val_entries,test_entries,val_bool,test_bool,val_mat,test_mat=test_val_loader(hard_gen,train_in)
        assert train_in[test_bool].sum()==0
        assert train_in[val_bool].sum()==0
        return name,train_in,known_mask,val_entries,test_entries,val_bool,test_bool,val_mat,test_mat
    else:
        val_fold_in,val_report,val_fold_in_bool,val_report_bool,test_fold_in,test_report,test_fold_in_bool,test_report_bool=test_val_loader(hard_gen,train_in)
        return name,train_in,known_mask,val_fold_in,val_report,val_fold_in_bool,val_report_bool,test_fold_in,test_report,test_fold_in_bool,test_report_bool
  

In [47]:
parameters={'beta': 0.779,
 'drop_out_rate': 0.132,
 'latent_dim': 15,
 'lr': 0.00013}

**VAE-sigmoid without correction vs. MF**

In [48]:

hard_gen=False
soft_max=False
sampling=False
correction=False
name1,train_in,known_mask,val_entries,test_entries,val_bool,test_bool,val_mat,test_mat=name_load(hard_gen,soft_max,sampling,correction)

Evaluation Mode: easy_gen_sigmoid_nocorrection_no_sampling


In [53]:
recall_list_1,neg_recall_list_1,hit_list_1,vae_1=load_pred(name1)
print(name1)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


Test recall:0.6386643481443509
Test specificity:0.5016177545890661
Test precision:0.7464510520590322
easy_gen_sigmoid_nocorrection_no_sampling


In [54]:
recall_list_mf,neg_recall_list_mf,hit_list_mf=load_pred("mf",mf=True)

C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


Test recall:0.537495489584306
Test specificity:0.47231367600220064
Test precision:0.6969877583436009


In [55]:
comparison(recall_list_1,recall_list_mf,neg_recall_list_1,neg_recall_list_mf,hit_list_1,hit_list_mf)

recall: Ttest_relResult(statistic=8.843816266531107, pvalue=1.5331809141008722e-18)
specificity: Ttest_relResult(statistic=1.8060603649456421, pvalue=0.07109854077880035)
precision: Ttest_indResult(statistic=4.328196088193042, pvalue=1.531838868096496e-05)


**VAE-sigmoid with correction vs. MF**

In [77]:
correction=True
hard_gen=False
soft_max=False
sampling=False
name2,train_in,known_mask,val_entries,test_entries,val_bool,test_bool,val_mat,test_mat=name_load(hard_gen,soft_max,sampling,correction)

Evaluation Mode: easy_gen_sigmoid_corrected_no_sampling


In [78]:
recall_list_2,neg_recall_list_2,hit_list_2,vae_2=load_pred(name2)
print(name2)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars


Test recall:0.7139755674441481
Test specificity:0.5107054148037755
Test precision:0.7688982488626313
easy_gen_sigmoid_corrected_no_sampling


In [ ]:
recall_list_mf,neg_recall_list_mf,hit_list_mf=load_pred("mf",mf=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


Test recall:0.537495489584306
Test specificity:0.47231367600220064
Test precision:0.6969877583436009


In [60]:
comparison(recall_list_2,recall_list_mf,neg_recall_list_2,neg_recall_list_mf,hit_list_2,hit_list_mf)

recall: Ttest_relResult(statistic=15.837626451538037, pvalue=2.361501259660995e-54)
specificity: Ttest_relResult(statistic=2.35870777479238, pvalue=0.0184594798292878)
precision: Ttest_indResult(statistic=6.5037987691068055, pvalue=8.543855101749583e-11)


**Effect of sampling: VAE-sigmoid easy generalisation task with correction**

In [ ]:
correction=True
sampling=True
hard_gen=False
soft_max=False
name4,train_in,known_mask,val_entries,test_entries,val_bool,test_bool,val_mat,test_mat=name_load(hard_gen,soft_max,sampling,correction)
recall_list_4,neg_recall_list_4,hit_list_4,vae_4=load_pred(name4)
print(name4)

Evaluation Mode: easy_gen_sigmoid_corrected_sampled

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars


Test recall:0.6640636285197042
Test specificity:0.5713222948878687
Test precision:0.7797279441562427
easy_gen_sigmoid_corrected_sampled


In [65]:
hard_gen=False
sampling=False
correction=True
soft_max=False
name5,train_in,known_mask,val_entries,test_entries,val_bool,test_bool,val_mat,test_mat=name_load(hard_gen,soft_max,sampling,correction)
recall_list_5,neg_recall_list_5,hit_list_5,vae_5=load_pred(name5)
print(name5)

Evaluation Mode: easy_gen_sigmoid_corrected_no_sampling

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars


Test recall:0.7139755674441481
Test specificity:0.5107054148037755
Test precision:0.7688982488626313
easy_gen_sigmoid_corrected_no_sampling


In [ ]:
comparison(recall_list_4,recall_list_5,neg_recall_list_4,neg_recall_list_5,hit_list_4,hit_list_5)

recall: Ttest_relResult(statistic=-7.45035846227843, pvalue=1.2052303059460574e-13)
specificity: Ttest_relResult(statistic=6.173744041759997, pvalue=8.455161201998364e-10)
precision: Ttest_indResult(statistic=1.0522782947446474, pvalue=0.29271786442978087)


**Effect of Sampling: VAE-sigmoid hard generalisation task with correction**

In [ ]:
correction=True
sampling=False
hard_gen=True
soft_max=False
name6,train_in,known_mask,val_fold_in,val_report,val_fold_in_bool,val_report_bool,test_fold_in,test_report,test_fold_in_bool,test_report_bool=name_load(hard_gen,soft_max,sampling,correction)
recall_list_6,neg_recall_list_6,hit_list_6,vae_6=load_pred(name6,hard_gen=True)
print(name6)

Evaluation Mode: hard_gen_sigmoid_corrected_no_sampling

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Test recall:0.814877426507503
Test specificity:0.5762532051004097
Test precision:0.7898134076325567
hard_gen_sigmoid_corrected_no_sampling


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars


In [ ]:
correction=True
sampling=True
hard_gen=True
soft_max=False
name7,train_in,known_mask,val_fold_in,val_report,val_fold_in_bool,val_report_bool,test_fold_in,test_report,test_fold_in_bool,test_report_bool=name_load(hard_gen,soft_max,sampling,correction)
recall_list_7,neg_recall_list_7,hit_list_7,vae_7=load_pred(name7,hard_gen=True)
print(name7)

Evaluation Mode: hard_gen_sigmoid_corrected_sampled

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Test recall:0.751631389975218
Test specificity:0.64570726639469
Test precision:0.8202575867973243
hard_gen_sigmoid_corrected_sampled


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars


In [ ]:
comparison(recall_list_6,recall_list_7,neg_recall_list_6,neg_recall_list_7,hit_list_6,hit_list_7)

recall: Ttest_relResult(statistic=3.4946226866049455, pvalue=0.0005332777994444515)
specificity: Ttest_relResult(statistic=-2.169558978583597, pvalue=0.03103460495050768)
precision: Ttest_indResult(statistic=-1.2493113039559953, pvalue=0.211955536087552)


**Effect of Sampling: VAE-softmax hard generalisation task without correction**

In [ ]:
correction=False
hard_gen=True
sampling=False
soft_max=True
name8,train_in,known_mask,val_fold_in,val_report,val_fold_in_bool,val_report_bool,test_fold_in,test_report,test_fold_in_bool,test_report_bool=name_load(hard_gen,soft_max,sampling,correction)
recall_20_list_8,recall_50_list_8,recall_100_list_8=load_pred(name8,mf=False,hard_gen=True,soft_max=True)
print(name8)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide


recall@20:0.3441278684465418
recall@50:0.4393811356819726
recall@100:0.5232486234353879
hard_gen_soft_max_nocorrection_no_sampling


In [ ]:
correction=False
hard_gen=True
sampling=True
soft_max=True
name9,train_in,known_mask,val_fold_in,val_report,val_fold_in_bool,val_report_bool,test_fold_in,test_report,test_fold_in_bool,test_report_bool=name_load(hard_gen,soft_max,sampling,correction)
recall_20_list_9,recall_50_list_9,recall_100_list_9=load_pred(name9,mf=False,hard_gen=True,soft_max=True)
print(name9)

hard_gen_soft_max_nocorrection_sampled

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide


recall@20:0.3168542464504973
recall@50:0.4163602632977888
recall@100:0.48468431251425426


In [ ]:
comparison(recall_20_list_8,recall_20_list_9,recall_50_list_8,recall_50_list_9,recall_100_list_8,recall_100_list_9,soft_max=True)

recall@20: Ttest_relResult(statistic=1.6357543451015173, pvalue=0.10275536798998279)
recall@50: Ttest_relResult(statistic=1.341304502956455, pvalue=0.18065788423343307)
recall@100: Ttest_relResult(statistic=2.1449538998387307, pvalue=0.0326178800654658)


**Effect of side information: VAE-sigmoid easy generalisation task without correction or sampling**

In [ ]:
correction=False
sampling=False
hard_gen=False
soft_max=False
name11,train_in,known_mask,val_entries,test_entries,val_bool,test_bool,val_mat,test_mat=name_load(hard_gen,soft_max,sampling,correction,pretrained=True)
recall_list_11,neg_recall_list_11,hit_list_11,vae_11=load_pred(name11,hard_gen=False)
print(name11)

Evaluation Mode: easy_gen_sigmoid_nocorrection_no_sampling_pretrained

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars


Test recall:0.6511300241573013
Test specificity:0.5189819098630574
Test precision:0.7605831116869459
easy_gen_sigmoid_nocorrection_no_sampling_pretrained


In [71]:
hard_gen=False
sampling=False
correction=True
soft_max=False
name5,train_in,known_mask,val_entries,test_entries,val_bool,test_bool,val_mat,test_mat=name_load(hard_gen,soft_max,sampling,correction)
recall_list_5,neg_recall_list_5,hit_list_5,vae_5=load_pred(name5)
print(name5)

Evaluation Mode: easy_gen_sigmoid_corrected_no_sampling

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
C:\Users\helen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars


Test recall:0.7139755674441481
Test specificity:0.5107054148037755
Test precision:0.7688982488626313
easy_gen_sigmoid_corrected_no_sampling


In [ ]:
comparison(recall_list_5,recall_list_11,neg_recall_list_5,neg_recall_list_11,hit_list_5,hit_list_11)

recall: Ttest_relResult(statistic=6.893051124676405, pvalue=6.6080197054983125e-12)
specificity: Ttest_relResult(statistic=-0.6331077412274727, pvalue=0.5267546200058721)
precision: Ttest_indResult(statistic=0.798608537235049, pvalue=0.42455199363506824)


**Effect of side information: VAE-softmax hard generalisation task without correction or sampling**

In [ ]:
correction=False
hard_gen=True
sampling=False
soft_max=True
name12,train_in,known_mask,val_fold_in,val_report,val_fold_in_bool,val_report_bool,test_fold_in,test_report,test_fold_in_bool,test_report_bool=name_load(hard_gen,soft_max,sampling,correction,pretrained=True)
recall_20_list_12,recall_50_list_12,recall_100_list_12=load_pred(name12,mf=False,hard_gen=True,soft_max=True)
print(name12)

Evaluation Mode: hard_gen_soft_max_nocorrection_no_sampling_pretrained

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide


recall@20:0.298105838725882
recall@50:0.3800729357960792
recall@100:0.45124982438062183
hard_gen_soft_max_nocorrection_no_sampling_pretrained


In [ ]:
correction=False
hard_gen=True
sampling=False
soft_max=True
name8,train_in,known_mask,val_fold_in,val_report,val_fold_in_bool,val_report_bool,test_fold_in,test_report,test_fold_in_bool,test_report_bool=name_load(hard_gen,soft_max,sampling,correction)
recall_20_list_8,recall_50_list_8,recall_100_list_8=load_pred(name8,mf=False,hard_gen=True,soft_max=True)
print(name8)

Evaluation Mode: hard_gen_soft_max_nocorrection_no_sampling

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide


recall@20:0.3441278684465418
recall@50:0.4393811356819726
recall@100:0.5232486234353879
hard_gen_soft_max_nocorrection_no_sampling


In [ ]:
comparison(recall_20_list_8,recall_20_list_12,recall_50_list_8,recall_50_list_12,recall_100_list_8,recall_100_list_12,soft_max=True)

recall@20: Ttest_relResult(statistic=2.4217609348996345, pvalue=0.015933459174520742)
recall@50: Ttest_relResult(statistic=3.295272518581719, pvalue=0.0010798728050247972)
recall@100: Ttest_relResult(statistic=4.135382338798507, pvalue=4.403645551300788e-05)


### References
[1] Dawen Liang. dawenl/vae_cf. Retrieved September 11, 2020 from https://github.com/dawenl/vae_cf